<h5> Installing requirements </h5>

In [ ]:
! pip install azure-identity
! pip install azure-purview-scanning
! pip install azure-purview-administration
! pip install azure-purview-catalog
! pip install azure-purview-account
! pip install azure-purview-scanning
! pip install azure-core

<h5>Importing requirements to use them </h5>

In [ ]:
from azure.identity import ClientSecretCredential 
from azure.core.exceptions import HttpResponseError
from azure.purview.administration.account import PurviewAccountClient
from azure.purview.scanning import PurviewScanningClient

<h5>You will need to create a Service Principal if you don't have one yet:<h5>
<a>https://learn.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal</a>

You must assign the data source admin role to your service principal in the collection where you intend to register your source

In [ ]:
client_id = "<CLIENT_ID>"                                                   # The Service Principal ID.
client_secret = "<CLIENT_SECRET>"                                           # The Service Principal Secret.
tenant_id = "<TENANT_ID>"                                                   # Your Tenant ID.
reference_name_purview = "<PVIEW_ACCOUNT_NAME>"                             # Your Purview Account Name.
subscription_ID="<SUBSCRIPTION_ID>"                                         # Subscription ID where your SGBD is hosted.
rg_name = "<RESOURCE_GROUP_NAME"                                            # ResourceGroup name where your SGBD is hosted.
collection_name = "COLLECTION_NAME"                                         # The Purview's Collection Name where I'll include this source.
ds_name = "<DATA_SOURCE_REGISTRATION_NAME>"                                 # The datasource name. It'll appers on "Data Map" tab.
endpoint="***.postgres.database.azure.com"                                  # The endpoint of your SGBD.
resource_ID="You can get it by clicking 'JSON View' at Overview SGBD page"  # The SGBD Resource ID
location ="BrazilSouth"                                                     # The SGBD's location.


<h5>Functions to perform the authentication and clients creation </h5>

In [ ]:
# Here we are bringing together the variables above to pass they as parameter to create a credential object
def get_credentials():
	credentials = ClientSecretCredential(client_id=client_id, client_secret=client_secret, tenant_id=tenant_id)
	return credentials

#Here we are creating a function to authenticating on Purview's API and that returns a Purview's client to perform the registration
def get_purview_client():
	credentials = get_credentials()
	client = PurviewScanningClient(endpoint=f"https://{reference_name_purview}.scan.purview.azure.com", credential=credentials, logging_enable=True)  
	return client

#Here we are creating a function to authenticating on Purview's API (PurviewAccount) and that returns a Purview's client to verify the collection name
def get_admin_client():
	credentials = get_credentials()
	client = PurviewAccountClient(endpoint=f"https://{reference_name_purview}.purview.azure.com/", credential=credentials, logging_enable=True)
	return client

<h5>Prepareing data to perform the registration<h5>

In [ ]:

#Creating a client
try:
	admin_client = get_admin_client()
except ValueError as e:
        print(e)

# Adjusting the collection name if needed
collection_list = admin_client.collections.list_collections()
for collection in collection_list:
	if collection["friendlyName"].lower() == collection_name.lower():
		collection_name = collection["name"]

#Defining the kind of source by the endpoint
if endpoint.endswith(".mysql.database.azure.com"):
    kind='AzureMySQL'
    port=3306  #If you are using another port, change it. This is the default port
if endpoint.endswith(".postgres.database.azure.com"):
    kind='AzurePostgreSQL' 
    port=5432 #If you are using another port, change it. This is the default port
    


In [ ]:
#Creating the body to use in the API request
body_input = {
  "kind": kind,
  "name": ds_name,
  "properties": {
    "endpoint": endpoint,
    "location": "BrazilSouth",
    "port": port,
    "resourceGroup": rg_name,
    "resourceId": resource_ID,
    "resourceName": "mysqlFlexible",
    "serverEndpoint": endpoint,
    "subscriptionId": "17e1e007-40fb-49a2-a22b-ab5269dcd731",
    "collection": {
      "referenceName": collection_name,
      "type": "CollectionReference"
    }
  }
}

<h5> Finally, we are able to register the data source </h5>

In [ ]:
try:
	client = get_purview_client()
except ValueError as e:
        print(e)

try:
	response = client.data_sources.create_or_update(ds_name, body=body_input)
	print(response)
	print(f"Data source {ds_name} successfully created or updated")
except HttpResponseError as e:
    print(e)